<h1> Motivation: </h1>
<p>This notebook is designed to serve as supplementary material for my post on how to visuallize text data which can be read <span style="color:blue">[__HERE__.](https://www.sammywealth.com/blog/visualizing-text-data) </span>

Import the neccessary libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline


Read the data

In [2]:
df = pd.read_excel('MyArticle.xlsx')
df.tail()

,Document,Class,Source
50,"Historically, the political spectrum was...",PoliticalScience,https://learn.saylor.org/course/polsc108
51,"this unit, we will look at the state, a ...",PoliticalScience,https://learn.saylor.org/course/polsc109
52,This unit looks at the various forms of gove...,PoliticalScience,https://learn.saylor.org/course/polsc110
53,The Max Planck Manual has a global perspecti...,PoliticalScience,https://learn.saylor.org/course/polsc111
54,This unit traces the emergence of a world sy...,PoliticalScience,https://learn.saylor.org/course/polsc112


## Preprocess the data

We will now tokenize the documents. There are many libraries out there that we can use for this purpose. We could use SpaCy, nltk, TextBlob or even create our own function using regular expressions. However, would use `textblob` to tokenize these documents. I will also remove English stopwords and words made of less than 3 characters.

In [3]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')

In [4]:
from textblob import TextBlob

In [5]:
' '.join([word for word in TextBlob('How cool is Ame? reca came.\t on 10/15/2018!').words if (not word in my_stopwords and len(word) > 2)])

'How cool Ame reca came 10/15/2018'

In [6]:
df['CleanDoc'] = df['Document'].apply(lambda x: ' '.join([word.lower() for word in TextBlob(x).words if (not word.lower() in my_stopwords and len(word) > 2)]))

### Let's compare one cleaned document with its raw version

In [7]:
print(df.iloc[0,0])

The novel opens with Mrs. Bennet trying to persuade Mr. Bennet to visit Mr. Bingley, a rich and eligible bachelor who has arrived in the neighbourhood. After some verbal sparring with Mr. Bennet baiting his wife, it transpires that this visit has already taken place at Netherfield, Mr. Bingley's rented house. The visit is followed by an invitation to a ball at the local assembly rooms that the whole neighbourhood will attend.


In [8]:
print(df.iloc[0,3])

novel opens mrs bennet trying persuade bennet visit bingley rich eligible bachelor arrived neighbourhood verbal sparring bennet baiting wife transpires visit already taken place netherfield bingley rented house visit followed invitation ball local assembly rooms whole neighbourhood attend


### Great! Let's use the TfidfVectorizer of sklearn to transform our cleaned documents.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(min_df= 0, max_df = 0.04)

In [10]:
X = tv.fit_transform(df['CleanDoc'])
print(X.shape)
vocab = tv.get_feature_names()
print('Length of Vocabulary: {}'.format(len(vocab)))

(55, 1096)
Length of Vocabulary: 1096


### Convert the transformed documents to a matrix and export it to excel for use in Gephi

In [11]:
matrix = pd.DataFrame(X.toarray(), index = df['Class'], columns=vocab)

In [12]:
matrix.head()

,100,15,1565,1648,1830s,300,40,able,absorb,accept,...,work,working,works,worth,writes,written,york,young,younger,zones
Class,,,,,,,,,,,,,,,,,,,,,
PrideAndPrejudice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PrideAndPrejudice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PrideAndPrejudice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PrideAndPrejudice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PrideAndPrejudice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
matrix.to_excel('topics matrix.xlsx', index = True)

In [14]:
matrix.shape


(55, 1096)

<h2>If you are interested in the Gephi tutorial part only, <span style="color:red"> STOP HERE AND GO TO GEPHI.</span> otherwise <span style="color:BLUE"> KEEP READING.</span></h2>
<br></br>
<p></p>
<h2> The following section is about <span style="color:GREEN"> HOW TO VISUALIZE LDA TOPIC MODELS.</span></h2>
<p> There are many algorithms for doing topic modelling but <span style="color:blue"> gensim</span> is one of the most common ones. <p>

In [15]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Since we had already preprocessed out text above, all we need to do now is [tokenize](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html) the cleaned documents.

In [16]:
df['Tokenized'] = df['CleanDoc'].apply(lambda x: x.split())
df.head()

,Document,Class,Source,CleanDoc,Tokenized
0,The novel opens with Mrs. Bennet trying to per...,PrideAndPrejudice,https://en.wikipedia.org/wiki/Pride_and_Prejudice,novel opens mrs bennet trying persuade bennet ...,"[novel, opens, mrs, bennet, trying, persuade, ..."
1,"At the ball, Mr. Bingley is open and cheerful,...",PrideAndPrejudice,https://en.wikipedia.org/wiki/Pride_and_Prejudice,ball bingley open cheerful popular guests appe...,"[ball, bingley, open, cheerful, popular, guest..."
2,"When Jane visits Miss Bingley, she is caught i...",PrideAndPrejudice,https://en.wikipedia.org/wiki/Pride_and_Prejudice,jane visits miss bingley caught rain shower wa...,"[jane, visits, miss, bingley, caught, rain, sh..."
3,"Mr. Collins, a cousin of Mr. Bennet and heir t...",PrideAndPrejudice,https://en.wikipedia.org/wiki/Pride_and_Prejudice,collins cousin bennet heir longbourn estate vi...,"[collins, cousin, bennet, heir, longbourn, est..."
4,Elizabeth and her family meet the dashing and ...,PrideAndPrejudice,https://en.wikipedia.org/wiki/Pride_and_Prejudice,elizabeth family meet dashing charming george ...,"[elizabeth, family, meet, dashing, charming, g..."


<h3>They are 3 main steps in creating a gensim corpus.</h3>

<ul>
    <li>Create Corpus: This is a list of list of tokenized documents. This can easily become too large.</li>
    <li>Create token dictionary: This dictionary assignes an index to each token in the entire corpus of documents..</li>
    <li>Term Document Frequency: stores the index wordcount pair of each token in the token dictionary.</li>
</ul>

In [17]:
import gensim.corpora as corpora

# Create Corpus
texts = df['Tokenized']

# Create Dictionary
id2word = corpora.Dictionary(df['Tokenized'])

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 3), (30, 1), (31, 1)]]


## Train an LDA model.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### Get the most dominant topic for each document.

In [19]:
my_topics = [sorted(lda_model[bow][0], key = lambda x: x[1], reverse = True)[0][0] for bow in corpus]
[(i, my_topics.count(i)) for i in set(my_topics)]
df['PredictedTopic'] = ['Topic {}'.format(topicNumber + 1) for topicNumber in my_topics]
mat =pd.DataFrame(X.toarray(), index=my_topics, columns = vocab)
mat.to_excel('mat5.xlsx')
[(i, my_topics.count(i)) for i in set(my_topics)]

[(0, 18), (1, 13), (2, 6), (3, 18)]

Checkout the distribution of topics in our various classes.

In [20]:
df.groupby(by = ['Class', 'PredictedTopic']).agg({'Document': 'count'})

Document
Class             PredictedTopic          
PoliticalScience  Topic 1                9
                  Topic 3                1
                  Topic 4                2
PrideAndPrejudice Topic 1                6
                  Topic 2                3
                  Topic 3                4
                  Topic 4                3
StockTrading      Topic 3                1
                  Topic 4               13
Weather           Topic 1                3
                  Topic 2               10

It appears that political science is distributed between topics 1, 3 and 4 while PrideAndPrejudice is in all topics. Stock trading and weather seem to be unique to topics 4 and 2.
<p>Any one who has read [Pride And Prejudice by Jane Austen](https://en.wikipedia.org/wiki/Pride_and_Prejudice) will agree that it is a mixture of politics and weather and wealth.</p>

#### Let's retire to seeing the topics. First I get the top 40 words that define each topic and then I use the amzing `pyLDAvis` library to plot the topics.

In [21]:
for topic in lda_model.show_topics(formatted=False, num_words=40):
    print('Topic {}'.format(str(topic[0]+1))+ ': ' + '\n'+' | '.join([word[0] for word in topic[1]]) + '\n'+ '--'*78)

Topic 1: 
political | states | unit | state | government | course | perspective | elizabeth | politics | united | darcy | issues | look | forms | contemporary | world | others | ideologies | end | general | across | one | questions | jane | principles | study | science | various | institutions | role | global | century | twentieth | seen | creation | learn | first | governance | among | populations
------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 2: 
weather | earth | temperature | surface | different | air | lower | differences | system | due | one | pressure | solar | systems | occur | atmosphere | scale | changes | large | sun | climate | atmospheric | phenomena | effect | causes | affect | years | known | higher | part | spot | moisture | sunlight | cell | jet | stream | altitudes | cause | however | caused
---------------------------------------------------------------

In [22]:
for topic in lda_model.show_topics(formatted=False, num_words=40):
    print(topic)

(0, [('political', 0.030811898), ('states', 0.020873385), ('unit', 0.020143062), ('state', 0.017383704), ('government', 0.0130092), ('course', 0.010391863), ('perspective', 0.009877052), ('elizabeth', 0.009265046), ('politics', 0.00878077), ('united', 0.008147428), ('darcy', 0.007319779), ('issues', 0.0064352266), ('look', 0.006275257), ('forms', 0.006274406), ('contemporary', 0.005577479), ('world', 0.005574635), ('others', 0.00495894), ('ideologies', 0.004958465), ('end', 0.0046034767), ('general', 0.00459834), ('across', 0.004597168), ('one', 0.004315701), ('questions', 0.0042195744), ('jane', 0.0040715877), ('principles', 0.0040077767), ('study', 0.004007536), ('science', 0.003983518), ('various', 0.0037095603), ('institutions', 0.0037095598), ('role', 0.003708071), ('global', 0.0036281978), ('century', 0.003562005), ('twentieth', 0.003562005), ('seen', 0.0035601829), ('creation', 0.0035514492), ('learn', 0.0035409431), ('first', 0.0035409431), ('governance', 0.0035404793), ('among

In [23]:
from gensim.models import CoherenceModel
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.281403321042627

Coherence Score:  0.6346605815698426


In [24]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.175475  0.035312       1        1  34.990944
1     -0.070137 -0.160629       2        1  29.249521
3     -0.125897  0.136825       3        1  25.894949
2      0.020559 -0.011508       4        1   9.864580, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
639   Default  25.000000      weather  25.000000  30.0000  30.0000
1027  Default  27.000000    political  27.000000  29.0000  29.0000
1006  Default  18.000000       states  18.000000  28.0000  28.0000
1107  Default  18.000000         unit  18.000000  27.0000  27.0000
379   Default   9.000000        stock   9.000000  26.0000  26.0000
631   Default  17.000000        state  17.000000  25.0000  25.0000
1193  Default   9.000000     economic   9.000000  24.0000  24.0000
428   Default   4.000000         make   4.000000  23.0000  23.0000
1149  Default  12.000000   government  12.000000  22.0000  22.0000
618   Default   8.000000        earth   8.000000  21.0000  21.0000
413   Default   7.000000       shares   7.000000  20.0000  20.0000
634   Default   8.000000  temperature   8.000000  19.0000  19.0000
686   Default   8.000000      surface   8.000000  18.0000  18.0000
1237  Default   3.000000        sudan   3.000000  17.0000  17.0000
42    Default   9.000000    elizabeth   9.000000  16.0000  16.0000
1255  Default   6.000000     european   6.000000  15.0000  15.0000
1016  Default   9.000000       course   9.000000  14.0000  14.0000
367   Default   6.000000    investors   6.000000  13.0000  13.0000
99    Default   7.000000          led   7.000000  12.0000  12.0000
948   Default  10.000000  perspective  10.000000  11.0000  11.0000
40    Default   8.000000        darcy   8.000000  10.0000  10.0000
644   Default   7.000000          air   7.000000   9.0000   9.0000
393   Default   6.000000      company   6.000000   8.0000   8.0000
414   Default   6.000000       stocks   6.000000   7.0000   7.0000
410   Default   6.000000        price   6.000000   6.0000   6.0000
1028  Default   8.000000     politics   8.000000   5.0000   5.0000
519   Default   7.000000        lower   7.000000   4.0000   4.0000
580   Default   5.000000       people   5.000000   3.0000   3.0000
1007  Default   7.000000       united   7.000000   2.0000   2.0000
402   Default  12.000000       issues  12.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
1228   Topic4   1.615043       manual   2.124398   2.0421  -5.0305
1227   Topic4   1.615043  legislature   2.124398   2.0421  -5.0305
1225   Topic4   1.615043         hope   2.124398   2.0421  -5.0305
1224   Topic4   1.615043      federal   2.124398   2.0421  -5.0305
1238   Topic4   1.615043      unitary   2.124398   2.0421  -5.0305
1222   Topic4   1.615043        chose   2.124398   2.0421  -5.0305
1221   Topic4   1.615043       anyway   2.124398   2.0421  -5.0305
428    Topic4   3.489406         make   4.841280   1.9888  -4.2601
587    Topic4   1.034920        still   1.551981   1.9110  -5.4755
356    Topic4   1.844005        would   3.052264   1.8123  -4.8979
13     Topic4   1.787252        local   2.992270   1.8009  -4.9291
209    Topic4   0.740942         lady   1.253849   1.7902  -5.8097
200    Topic4   0.740942    catherine   1.253849   1.7902  -5.8097
569    Topic4   0.724126        firms   1.242129   1.7766  -5.8326
575    Topic4   0.724126       makers   1.242129   1.7766  -5.8326
825    Topic4   1.615044         lead   2.820703   1.7586  -5.0305
448    Topic4   0.710191         tend   1.247858   1.7526  -5.8520
437    Topic4   0.731503    difficult   1.415459   1.6561  -5.8225
1019   Topic4   1.493292      explore   3.021383   1.6115  -5.1088
962    Topic4   1.491968      central   3.022506   1.6102  -5.1097
1124   Topic4   1.480626  governments   3.027602   1.6009  -5.1174
1141   Topic4  

<p> <span style="color:blue"> [Stay Updated! Always checkout my blog for more posts!](https://www.sammywealth.com/blog/visualizing-text-data) </span> </p>